In [35]:
import os, enum;
from abc import ABC, abstractmethod
def addFiles(fileList, directory):
   for root, dirs, files in os.walk(directory):
        for file in files:
            fileList.append(root+'/'+file)
        for subdir in dirs:
            addFiles(fileList, subdir);
            
    
class Result(enum.Enum):
    failedMatch = 0           #didn't match - reset to zero
    matchedProgress = 1       #all requirements met - move on to the next requirement 
    oneOfManyMatches = 2      #Data gathered - keep on feeding me more lines
    gameSetMatch = 3          #Found end of data set - go spit out results
"""
    define an array of matching patterns
    Params:
    1 - Index to expect parameter 2
    2 - Space delimited match string
    3 - Data array index to capture
 - Populated value - starts as None (null)
"""
class Requirement(ABC):
    @abstractmethod
    def testline(self, line):
        pass
    @abstractmethod
    def reset(self):
        pass

class Splicer(Requirement):
    def __init__(self, matchIdx, matchValue, captureIndex, foundValue):
        self.matchIdx = matchIdx
        self.matchValue = matchValue
        self.captureIndex = captureIndex
        self.foundValue = foundValue
    @abstractmethod
    def getSpliceChar(self):
        pass
    def testline(self, line):
        sp = line.split(getSpliceChar())
        if (sp is not None and 
            len(sp) > matchIdx and 
            len(sp) > captureIndex and 
            matchValue == sp[matchIdx]):
            foundValue = sp[captureIndex]
            return Result.matchedProgress
        return Result.failedMatch
    def reset(self):
        self.foundValue = None
        
class SpaceSplicer(Splicer):
    def getSpliceChar(self):
        return ' '

class ColonSplicer(Splicer): #Ouch?
    def getSpliceChar(self):
        return ':'
class Blank(Requirement):
    def reset(self):
        return;
    def testline(self, line):
        if (line is None):
            print('Not sure what to: do with None-zies')
        else: 
            if (len(strip(line)) == 0):
                #blank line!
                return Result.matchedProgress
        return Result.failedMatch
                
            
class RequirementSet:
    reqArray = []
    reqIndex = 0;
    def __init__(self):
        reqArray.append( SpaceSplicer(0, 'commit', 1, None) )
        reqArray.append( ColonSplicer(0, 'Author', 1, None) )
        reqArray.append( ColonSplicer(0, 'Date', 1, None) )
    def reset(self):
        for req in reqArray:
            req.reset();
    def processResult(self, resultStream, rslt):
            match rslt:
                case Result.failedMatch:
                    reset();
                    reqIndex = 0;
                case Result.matchedProgress:
                    reqIndex += 1
                    if (reqIdx >= len(reqArray)):
                        print('ERROR - Last element of RequirementSet cannot return Result.matchedProgress');
                        reset();
                        reqIndex = 0;
                case Result.oneOfManyMatches:
                    #Just keep reading until done
                case Result.gameSetMatch:
                    for req in reqArray:
                        req.printResults(resultStream)
                case _:
                    print('ERROR - Unknown result type: '+rslt)
    def testline(self, line, resultStream):
        if (reqIndex >= len(reqArray)):
            print('ERROR - RequirementIndex out of range!')
            reset();
            reqIndex = 0;
        else:
            #needs SWITCH
            processResult(resultStream, reqArray[reqIndex].testline(line))
                        
                

fileList = []
reqSet = RequirementSet()
addFiles(fileList, '/home/matt/Projects/Web3HackerNetwork/data/github');
print('Found '+str(len(fileList))+' files');
root = '/home/matt/Projects/Web3HackerNetwork/data/samples'
log = open(root+'/commitScan.log', 'w')
result = open(root+'/commitScan.tsv', 'w')
for fileName in fileList:
    firstline = None
    fileExt = None
    linecount = 0
    commitLineIdx = 0
    commitLine = None
    commitID = None

    
    
    with open(fileName, 'r') as file:
        log.write('Reading file: '+fileName+'\n')
        
        for line in file:
            linecount += 1
            reqSet.testline(line, result);
        file.close()
    
    
    print(str(file)+' : '+str(linecount)+' lines')
    for req in reqArray:
        print('\t'+str(req.matchValue)+'->'+str(req.foundValue));
    

SyntaxError: invalid syntax (757228822.py, line 84)